In [197]:
%load_ext autoreload
%autoreload 2

from llm_service import litellm_service, message_parse, custom_llm_service
from auto_eval import extract_valid_json
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
multi = {

        "index": 30,
        "category": "Popular science",
        "question": "I walk to my friend's house, averaging 3mph. How fast would I have to run back to double my average speed for the entire trip?",
        "human_answer": "Instantaneously fast, it is mathematically impossible.",
        "correct_answer": "Infinitely fast",
        "multiple_choice": [
            "Infinitely fast",
            "6mph",
            "9mph",
            "12mph"
        ]
        
}

random.shuffle(multi["multiple_choice"])
letters = ['A', 'B', 'C', 'D']
correct_answer_idx = multi["multiple_choice"].index(multi["correct_answer"])
correct_letter = letters[correct_answer_idx]
answers = '\n'.join([f'{letter}. {question}' for letter, question 
                     in zip(letters, multi["multiple_choice"])])
random_letter = random.choice(letters)

prompt = f"""QUESTION
{multi["question"]}

ANSWERS
{answers}

Provide an explanation for your thinking and then select a single choice from ANSWERS that answer the QUESTION. Return in JSON format, for example:
{{"ANSWER": "{random_letter}"}}
"""
print(prompt)
print('-------------------\n')
print(f"Correct answer: {correct_letter}")

QUESTION
I walk to my friend's house, averaging 3mph. How fast would I have to run back to double my average speed for the entire trip?

ANSWERS
A. 6mph
B. Infinitely fast
C. 9mph
D. 12mph

Provide an explanation for your thinking and then select a single choice from ANSWERS that answer the QUESTION. Return in JSON format, for example:
{"ANSWER": "D"}

-------------------

Correct answer: B


In [200]:
llm_query = litellm_service()
#llm_query = custom_llm_service()

messages = [{ "content": f'{prompt}',"role": "user"}]

models = ["gpt-4-turbo-preview", "meta.llama3-70b-instruct-v1:0", "command-r", "mistral/mistral-large-latest", "mistral/open-mixtral-8x22b", "claude-3-opus-20240229", "vertex_ai/gemini-1.5-pro", "vertex_ai/gemini-1.0-pro"]
hyperparameters = {
    "temperature": 0, 
    "max_tokens": 2048,
    "num_retries": 2,
}

response = llm_query.completion(model="vertex_ai/gemini-1.5-pro", messages=messages, **hyperparameters)

answer = message_parse(response)
print(answer)

print('------ json output -------')

valid_json = extract_valid_json(answer)
print(valid_json)
if valid_json:
    print('\n------ answer ------')
    print(f"Correct Answer: {correct_letter}")
    print(f"Correct: {valid_json['ANSWER'] == correct_letter}")
    print('\n*** Choices ***')
    print(answers)

```json
{
 "ANSWER": "B",
 "Explanation": "Here's why:  \n\n* **Average Speed Isn't Just the Mean:** Average speed is calculated as total distance divided by total time. It's not simply the average of the two speeds. \n* **The Impact of Time:**  Since you're traveling the same distance, to double your average speed, you would need to halve the total time. However, you've already spent a certain amount of time walking to your friend's house. \n* **Impossible to Make Up:** No matter how fast you run back, you can't make up for the time already spent walking. To achieve double the average speed, you'd need to return in zero time, which is impossible. \n\nTherefore, you would have to run back infinitely fast, which is not practically achievable."
}
``` 

------ json output -------
{'ANSWER': 'B', 'Explanation': "Here's why:  \n\n* **Average Speed Isn't Just the Mean:** Average speed is calculated as total distance divided by total time. It's not simply the average of the two speeds. \n* **